In [1]:
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Softmax, Dense, Lambda, BatchNormalization, ReLU
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# Convert y_train into one-hot format
temp = []
for i in range(len(y_train)):
    temp.append(to_categorical(y_train[i], num_classes=10))
y_train = np.array(temp)
# Convert y_test into one-hot format
temp = []
for i in range(len(y_test)):    
    temp.append(to_categorical(y_test[i], num_classes=10))
y_test = np.array(temp)

In [4]:
#reshaping
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [5]:
inputs = Input(shape=(28,28,1))
out = Lambda(lambda x: x/1000)(inputs)
out = Conv2D(4, 3, use_bias=False)(out)
out = BatchNormalization()(out)
out = ReLU()(out)
# out = Lambda(lambda x: x**2+x)(out)
out = AveragePooling2D()(out)
# out = Lambda(lambda x: x*4)(out)
out = Conv2D(8, 3, use_bias=False)(out)
out = BatchNormalization()(out)
out = ReLU()(out)
# out = Lambda(lambda x: x**2+x)(out)
out = AveragePooling2D()(out)
# out = Lambda(lambda x: x*4)(out)
out = Flatten()(out)
out = Dense(10, activation=None)(out)
out = Softmax()(out)
model = Model(inputs, out)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
lambda (Lambda)              (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 4)         36        
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 4)         16        
_________________________________________________________________
re_lu (ReLU)                 (None, 26, 26, 4)         0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 13, 13, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 8)         288   

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    metrics=['acc']
    )

In [8]:
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3734 - acc: 0.8874 - val_loss: 0.1168 - val_acc: 0.9623
Epoch 2/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0823 - acc: 0.9748 - val_loss: 0.0758 - val_acc: 0.9772
Epoch 3/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0648 - acc: 0.9811 - val_loss: 0.0640 - val_acc: 0.9791
Epoch 4/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0591 - acc: 0.9815 - val_loss: 0.0520 - val_acc: 0.9835
Epoch 5/15
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0511 - acc: 0.9853 - val_loss: 0.0530 - val_acc: 0.9814
Epoch 6/15
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0472 - acc: 0.9857 - val_loss: 0.0537 - val_acc: 0.9835
Epoch 7/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0456 - acc: 0.9855 - val_loss: 0.0429 - val_acc: 0.9853
Epoch 8/15
1875/1875 [==============================] - 3s 1ms

In [9]:
X = X_test[0]
X.shape, X.min(), X.max()

((28, 28, 1), 0, 255)

In [10]:
model2 = Model(model.input, model.layers[-2].output)

In [11]:
y = model2.predict(X_test[[0]]) - model.layers[-2].weights[1].numpy()

In [12]:
print(len(model.weights))
for weights in model.weights:
    print(weights.shape)

12
(3, 3, 1, 4)
(4,)
(4,)
(4,)
(4,)
(3, 3, 4, 8)
(8,)
(8,)
(8,)
(8,)
(200, 10)
(10,)


In [13]:
gamma = model.layers[3].weights[0].numpy()
beta = model.layers[3].weights[1].numpy()
moving_mean = model.layers[3].weights[2].numpy()
moving_var = model.layers[3].weights[3].numpy()
epsilon = model.layers[3].epsilon

In [14]:
a1 = gamma/(moving_var+epsilon)**.5
b1 = beta-gamma*moving_mean/(moving_var+epsilon)**.5
a1, b1

(array([ 7.476623, 12.203958, 11.305573,  7.275524], dtype=float32),
 array([ 1.0641948 , -0.2928645 ,  0.96127605, -1.3354607 ], dtype=float32))

In [15]:
gamma = model.layers[7].weights[0].numpy()
beta = model.layers[7].weights[1].numpy()
moving_mean = model.layers[7].weights[2].numpy()
moving_var = model.layers[7].weights[3].numpy()
epsilon = model.layers[7].epsilon

In [16]:
a2 = gamma/(moving_var+epsilon)**.5
b2 = beta-gamma*moving_mean/(moving_var+epsilon)**.5
a2, b2

(array([1.8293172, 2.1094613, 2.1327975, 1.8025072, 2.0401976, 2.2244277,
        2.00088  , 1.0940925], dtype=float32),
 array([-0.56788176,  3.609695  ,  1.9935554 ,  0.6033936 ,  0.6063134 ,
        -1.7730187 ,  6.8325047 , -0.12943316], dtype=float32))

In [17]:
in_json = {
    "in": X.astype(int).flatten().tolist(), # X is already 1000 times to begin with
    "conv2d_1_weights": (model.layers[2].weights[0].numpy()*(10**3)).round().astype(int).flatten().tolist(),
    "conv2d_1_bias": (np.array([0]*4)*(10**3)**2).round().astype(int).flatten().tolist(),
    "bn_1_a": (a1*(10**3)).round().astype(int).flatten().tolist(),
    "bn_1_b": (b1*(10**3)**3).round().astype(int).flatten().tolist(),
    # average pooling will scale another 10**2 times
    "conv2d_2_weights": (model.layers[6].weights[0].numpy()*(10**3)).round().astype(int).flatten().tolist(),
    "conv2d_2_bias": (np.array([0]*8)*(10**(3*4+2))).round().astype(int).flatten().tolist(),
    "bn_2_a": (a2*(10**3)).round().astype(int).flatten().tolist(),
    "bn_2_b": (b2*(10**(3*5+2))).round().astype(int).flatten().tolist(),
    # average pooling will scale another 10**2 times
    "dense_weights":(model.layers[11].weights[0].numpy()*(10**3)).round().astype(int).flatten().tolist(),
    "dense_bias": np.zeros(model.layers[11].weights[1].numpy().shape).tolist() # zero because we are not doing softmax in circom, just argmax
}

In [18]:
out_json = {
    "scale": 10**-22,
    "out": y.flatten().tolist(),
    "label": int(y.argmax())
}
out_json

{'scale': 1e-22,
 'out': [-0.9809412956237793,
  -2.2749931812286377,
  6.040533065795898,
  9.122146606445312,
  -11.2403564453125,
  -0.5773029327392578,
  -17.33807945251465,
  16.305007934570312,
  -1.445622205734253,
  2.890042781829834],
 'label': 7}

In [19]:
import json

In [20]:
with open("mnist_latest_precision_input.json", "w") as f:
    json.dump(in_json, f)

In [21]:
with open("mnist_latest_precision_output.json", "w") as f:
    json.dump(out_json, f)